### Part  1: Scrape Mars News

In [38]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import json
import pymongo
#from pymongo import MongoClient

In [39]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [40]:
# Mars News Site to Scrape
url = 'https://redplanetscience.com/'
browser.visit(url)

In [41]:
# Return all the HTML on page
html = browser.html

# Create a Beautiful Soup object, pass in our HTML, and call 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [42]:
# Scrape titles and teaser body text
#content_titles = soup.find_all('div', class_='content_title')
#previews = soup.find_all('div', class_='article_teaser_body')

In [43]:

elements = soup.find_all('div', class_='list_text')
elements

[<div class="list_text">
 <div class="list_date">September 26, 2022</div>
 <div class="content_title">The Detective Aboard NASA's Perseverance Rover</div>
 <div class="article_teaser_body">An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">September 26, 2022</div>
 <div class="content_title">My Culture, My Voice</div>
 <div class="article_teaser_body">In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">September 23, 2022</div>
 <div class="content_title">5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover</div>
 <div class="article_teaser_body">The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from fun

In [44]:
scraped_data = []

for x in elements:
    try:
        data = {}
        data['content_titles'] = x.find('div', class_='content_title').get_text()
        data['previews'] = x.find('div', class_='article_teaser_body').get_text()
        scraped_data.append(data)
 
        browser.links.find_by_partial_text('More').click()
 
    except Exception as E:
        print(E)   

scraped_data

[{'content_titles': "The Detective Aboard NASA's Perseverance Rover",
  'previews': 'An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.'},
 {'content_titles': 'My Culture, My Voice',
  'previews': 'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.'},
 {'content_titles': "5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover",
  'previews': "The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."},
 {'content_titles': "NASA's Mars Rover Drivers Need Your Help",
  'previews': 'Using an online tool to label Martian terrain types, you can train an artificial intelligence algorithm that could improve the way engineers guide the Curiosity rover.'},
 {'content_titles': "How NASA's Mars Helicopter Will Reach t

In [45]:
browser.quit()

##### Optional Activities

In [46]:
with open ('scraped_mars_news.json', 'w') as file:
    json.dump(scraped_data, file)

In [50]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the mars news database
db = client.marsnewsdb

#### Need to sort this out , not writing DB